In [37]:
import pandas as pd
import numpy as np
from bokeh.plotting import *
from bokeh.charts import Bar, Histogram, output_file, show, hplot
from bokeh.io import save, output_file, show
from bokeh.models import (
  GMapPlot, GMapOptions, DataRange1d, DataRange1d, ColumnDataSource, Circle, PanTool, WheelZoomTool, BoxSelectTool
)

In [26]:
NYC_taxi_data = pd.read_csv('Desktop/Taxi_Task/cleaned_NYC_taxi_data_with_geo.csv')
NYC_taxi_data.head()

,cartodb_id,the_geom,store_and_fwd_flag,payment_type,ratecodeid,passenger_count,vendorid,field_1,percentage_tipped,total_amount,...,pickup_latitude,pickup_longitude,trip_distance,tpep_dropoff_datetime,tpep_pickup_datetime,boroname,ntacode,ntaname,shape_area,shape_leng
0,20001,0101000020E61000009E07FB9F518052C06B9B0380BA5E...,False,1,1,1,2,22521,22.285714,9.36,...,40.728672,-73.994225,1.23,2002-06-15 11:50:00+00,2002-06-15 11:41:00+00,Manhattan,MN23,West Village,25000526.3764,29385.030411
1,20002,0101000020E6100000596DFE5FF57C52C02963F85F1965...,False,1,1,1,1,22522,16.666667,14.80,...,40.765697,-73.954773,2.20,2002-06-15 11:57:00+00,2002-06-15 11:41:00+00,Manhattan,MN33,East Harlem South,16651123.4152,18625.003445
2,20003,0101000020E61000004DFEFEFF057F52C0300A06005C62...,False,2,1,1,2,22523,0.000000,13.30,...,40.779320,-73.957817,2.42,2002-06-15 11:57:00+00,2002-06-15 11:41:00+00,Manhattan,MN14,Lincoln Square,15805638.3015,19866.052989
3,20004,0101000020E6100000BDCB0360557E52C0CB4500608A5F...,False,2,1,2,1,22524,0.000000,4.80,...,40.741791,-73.974915,0.40,2002-06-15 11:44:00+00,2002-06-15 11:41:00+00,Manhattan,MN20,Murray Hill-Kips Bay,14465955.0048,23135.103491
4,40001,0101000020E610000067A8FEFF077E52C02CD804208764...,False,2,1,4,1,45199,0.000000,11.30,...,40.769596,-73.951645,1.70,2004-06-15 16:05:00+00,2004-06-15 15:50:00+00,Manhattan,MN99,park-cemetery-etc-Manhattan,91540308.4145,155018.226038


In [30]:
# bar chart: mean tip amount by area
NYC_taxi_data_tip_greater_zero = NYC_taxi_data.loc[NYC_taxi_data['tip_amount'] > 0]

boro_by_tip_mean_1 = NYC_taxi_data_tip_greater_zero.groupby(['boroname'])['tip_amount'].mean()
boro_by_tip_count_1 = NYC_taxi_data_tip_greater_zero.groupby(['boroname'])['tip_amount'].count()

def plot_bar_chart_mean_tip_by_area(tip_mean, tip_count):
    
    data = {
        'sample size': list(pd.Series(tip_count).values),
        'area names': list(pd.Series(tip_mean).index),
        'mean tip amount': list(pd.Series(tip_mean).values)
    }

    bar = Bar(data, values='mean tip amount', label=['area names', 'sample size'],
               agg='mean', title='Mean Tip Amount By Area', width=400)

    output_file('area_by_tip_mean.html')
    show(hplot(bar))

plot_bar_chart_mean_tip_by_area(boro_by_tip_mean_1, boro_by_tip_count_1)

In [31]:
# bar chart: mean tip amount by area (on subset)
NYC_taxi_data_tip_subset = NYC_taxi_data_tip_greater_zero.loc[NYC_taxi_data['tip_amount'] < 15]

boro_by_tip_mean_2 = NYC_taxi_data_tip_subset.groupby(['boroname'])['tip_amount'].mean()
boro_by_tip_count_2 = NYC_taxi_data_tip_subset.groupby(['boroname'])['tip_amount'].count()

bar_chart_mean_tip_by_area(boro_by_tip_mean_2, boro_by_tip_count_2)

In [32]:
# histogram: tip amount (on subset)
def plot_hist_tip(tip):
    
    data = {
        'tip amount': list(tip)
    }

    hist = Histogram(data, values='tip amount', title='Tip Amount',  agg='count', width=400)

    output_file('hist.html')
    show(hplot(hist))

plot_hist_tip(NYC_taxi_data_tip_subset['tip_amount'])

In [43]:
# dropoff coordinates on google maps plot 
# color coding tip_amount

dropoff_lat = list(NYC_taxi_data['latitude'])
dropoff_lon = list(NYC_taxi_data['longitude']) 
colorcode = ["#%02x%02x%02x"% (int(r), 150, 150) for r in NYC_taxi_data['tip_amount']]

def plot_gmap_coordinates(latitude, longitude, color):

    map_options = GMapOptions(lat=40.82, lng=-74.023, map_type='roadmap', zoom=10)

    plot = GMapPlot(
        x_range=DataRange1d(), y_range=DataRange1d(), map_options=map_options, title='NYC'
    )

    source = ColumnDataSource(
        data={
            'lat':latitude,
            'lon':longitude,
            'col':colors
        }
    )
    circle = Circle(x='lon', y='lat', size=1, fill_color='blue', fill_alpha=0.2, line_color=None)
    plot.add_glyph(source, circle)
    
    circle = Circle(x='lon', y='lat', size=1, fill_color='col', fill_alpha=0.1, line_color=None)
    plot.add_glyph(source, circle)

    plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
    output_file("gmap_plot.html")
    show(plot)

plot_gmap_coordinates(dropoff_lat, dropoff_lon, colorcode) # with single color 

ERROR://anaconda/lib/python2.7/site-packages/bokeh/core/validation/check.pyc:E-1000 (COLUMN_LENGTHS): ColumnDataSource column lengths are not all the same: ColumnDataSource, ViewModel:ColumnDataSource, ref _id: fbb50f44-2fbd-4448-b3f4-ce3f42728705
